In [75]:
from pandas import read_excel
from pandas import DataFrame
from pandas import concat
from pandas import Series
import numpy
from numpy import append
from numpy import array
from numpy import diff
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.layers import Dropout
from sklearn.metrics import mean_squared_error
from math import sqrt
import warnings
warnings.filterwarnings("ignore")

In [76]:
# create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return Series(diff)

In [77]:
# invert differenced value
def inverse_difference(history, yhat, interval=1):
    return yhat + history[-interval]

In [78]:
# Get technical indicators
def get_technical_indicators(data):
    # Create Moving Average
    data = DataFrame(data)
    dataset = DataFrame()
    dataset['ma5'] = data.rolling(window=5).mean()
    dataset['ma20'] = data.rolling(window=25).mean()
    
    # Create MACD
    dataset['25ema'] = data.ewm(span=25).mean()
    dataset['10ema'] = data.ewm(span=10).mean()
    dataset['MACD'] = dataset['10ema']-dataset['25ema']

    # Create Bollinger Bands
    dataset['20sd'] = data.rolling(window = 20).std()
    dataset['upper_band'] = dataset['ma20'] + (dataset['20sd']*2)
    dataset['lower_band'] = dataset['ma20'] - (dataset['20sd']*2)
    
    # Create Exponential moving average
    dataset['ema'] = data.ewm(com=0.5).mean()
    
    # Create Momentum
    dataset['momentum'] = data-1
    # dataset['log_momentum'] = log(dataset['momentum'])
    return dataset.dropna()

In [79]:
# frame a sequence as a supervised learning problem
def timeseries_to_supervised(data, lag=1):
    df = DataFrame(data)
    columns = [df.shift(i) for i in range(1, lag+1)]
    columns.append(df)
    df = concat(columns, axis=1)
    df.fillna(0, inplace=True)
    return df

In [80]:
# scale train and test data to [-1, 1]
def scale(train, test):
    # fit scaler
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(train)
    # transform train
    train = train.reshape(train.shape[0], train.shape[1])
    train_scaled = scaler.transform(train)
    # transform test
    test = test.reshape(test.shape[0], test.shape[1])
    test_scaled = scaler.transform(test)
    return scaler, train_scaled, test_scaled

In [81]:
# inverse scaling for a forecasted value
def invert_scale(scaler, X, value):
    new_row = [x for x in X] + [value]
    array = numpy.array(new_row)
    array = array.reshape(1, len(array))
    inverted = scaler.inverse_transform(array)
    return inverted[0, -1]

In [82]:
# fit an LSTM network to training data
def model_fit(train, batch_size, nb_epoch, neurons):
    X, y = train[:, 0:-1], train[:, -1]
    X = X.reshape(X.shape[0], 1, X.shape[1])
    model = Sequential()
    model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    for i in range(nb_epoch):
        model.fit(X, y, epochs=1, batch_size=batch_size, verbose=1, shuffle=False)
        model.reset_states()
    return model

In [83]:
# make a one-step forecast
def model_predict(model, batch_size, X):
    X = X.reshape(1, 1, len(X))
    yhat = model.predict(X, batch_size=batch_size)
    return yhat[0,0]

In [84]:
# run a repeated experiment
def experiment(series, features):
    # transform data to be stationary
    raw_values = series.values
    diff_values = difference(raw_values, 1)

    # get technical indicators
    df = get_technical_indicators(diff_values)

    # transform data to be supervised learning
    supervised = timeseries_to_supervised(df, features)
    supervised_values = supervised.values
    
    # split data into train and test-sets
    len_test = 30
    train, test = supervised_values[:-len_test], supervised_values[-len_test:]
    
    # transform the scale of the data
    scaler, train_scaled, test_scaled = scale(train, test)
    
    # fit the base model
    # n_features = data.shape[1]-1
    n_batch = 1
    nb_epoch = 50
    n_neurons = 1
    mod = model_fit(train_scaled, n_batch, nb_epoch, n_neurons)

    # forecast test dataset
    predictions = list()
    for i in range(len(test_scaled)):
        # make one-step forecast
        X, y = test_scaled[i, 0:-1], test_scaled[i, -1]
        yhat = model_predict(mod, n_batch, X)
        # invert scaling
        yhat = invert_scale(scaler, X, yhat)
        # invert differencing
        yhat = inverse_difference(raw_values, yhat, len(test_scaled)+1-i)
        # store forecast
        predictions.append(yhat)

    return DataFrame(predictions)

In [ ]:
path = "C:\\Users\\fh20175\\Box\\Habbab Fatima\\Experiment set 3\\DataSets\\"
asset_list = read_excel("C:\\Users\\fh20175\\Box\\Habbab Fatima\\Experiment set 6\\asset_list.xlsx")
no_features = read_excel("C:\\Users\\fh20175\\Box\\Habbab Fatima\\Experiment set 6\\features.xlsx")
no_assets = len(asset_list)

predictions = DataFrame()

for i in range(0,no_assets):
    ticker = asset_list.iloc[i,0]
    data = read_excel(path+ticker+".xlsx")
    series = data[["Close"]].dropna()
    
    features = no_features.iloc[i,0]
    
    yhat = experiment(series, features)

    predictions[ticker] = yhat

  1/578 [..............................] - ETA: 14:51 - loss: 0.2031

In [ ]:
predictions.to_excel("pred30.xlsx")